<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#导入相关的包" data-toc-modified-id="导入相关的包-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>导入相关的包</a></span></li><li><span><a href="#导入数据" data-toc-modified-id="导入数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>导入数据</a></span></li><li><span><a href="#形成训练数据" data-toc-modified-id="形成训练数据-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>形成训练数据</a></span></li><li><span><a href="#训练模型" data-toc-modified-id="训练模型-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>训练模型</a></span></li></ul></div>

# 导入相关的包

In [22]:
import pandas as pd
import numpy as np
import xgboost as xgb
import time
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

# 导入数据

In [29]:
%%time
trains = pd.read_csv('../raw_data/d_train.csv',encoding="gbk")
tests = pd.read_csv("../raw_data/d_test_A.csv",encoding="gbk")
trains.drop(trains[trains["年龄"] >= 84].index,inplace=True)
fea_train = pd.read_csv("../raw_data/fea_train.csv")
fea_test = pd.read_csv("../raw_data/fea_test.csv")
fea_train1 = pd.read_csv("../raw_data/fea_train_1.csv")
fea_test1 = pd.read_csv("../raw_data/fea_test_1.csv")
trains = pd.merge(trains, fea_train, how="left",on="id")
trains = pd.merge(trains, fea_train1, how="left",on="id")
tests = pd.merge(tests, fea_test, how="left",on="id")
tests = pd.merge(tests, fea_test1, how="left",on="id")
trains["血糖"] = trains["血糖"].apply(lambda x: 1 if x > 10 else 0)
trains["血糖"] = trains["血糖"].astype(np.int32)

Wall time: 238 ms


# 形成训练数据


In [30]:
%%time
def make_feat(train,test):
    train_id = train.id.values.copy()
    test_id = test.id.values.copy()
    data = pd.concat([train,test])
    data['性别'] = data['性别'].map({'男': 1,'女': 0})
    # data["性别"] = data['性别'].astype(int)
    # data['date'] = (pd.to_datetime(data['date']) - parse('2017-10-09')).dt.days
    # data['体检日期'] = (pd.to_datetime(data['体检日期']) - parse('2017-10-09')).dt.days
    data.drop("体检日期",axis = 1,inplace= True)
    # data.fillna(value = -1, inplace = True)
    train_feat = data[data.id.isin(train_id)]
    test_feat = data[data.id.isin(test_id)]
    return train_feat,test_feat
train, test = make_feat(trains, tests)
predictors = [f for f in list(train.columns) if f not in ["血糖","blood_sugar","id","blood_sugar_log","体检日期"]]
# X_train, X_test, y_train,y_test = train_test_split(train[predictors], train["血糖"],test_size=0.1,random_state=42)
xgb_train = xgb.DMatrix(train[predictors], label=train["血糖"])
# xgb_eval = xgb.DMatrix(X_test, label=y_test)
xgb_test = xgb.DMatrix(test[predictors])

Wall time: 87.6 ms


In [31]:
train.shape
test.shape
test.head()
train.head()
(len(train["血糖"]) - sum(train["血糖"])) / sum(train["血糖"])

(5622, 93)

(1000, 93)

,*r-谷氨酰基转换酶,*丙氨酸氨基转换酶,*天门冬氨酸氨基转换酶,*总蛋白,*球蛋白,*碱性磷酸酶,ATSm,ATSs,feature_0_feature_1,feature_0_is_normal,...,红细胞平均血红蛋白量,红细胞计数,肌酐,血小板体积分布宽度,血小板平均体积,血小板比积,血小板计数,血糖,血红蛋白,高密度脂蛋白胆固醇
0,34.36,26.69,23.85,82.75,36.72,116.08,4.7700,19.0800,50.54,1,...,30.6,4.94,65.87,12.8,10.8,0.26,241.0,NaN,151.0,1.44
1,111.43,34.98,29.75,71.90,27.81,90.07,5.9500,23.8000,64.73,1,...,29.3,5.84,89.99,14.6,11.5,0.28,242.0,NaN,171.0,1.22
2,NaN,NaN,NaN,NaN,NaN,NaN,5.6388,22.5552,61.60,1,...,29.9,5.72,73.29,9.9,8.9,0.35,398.0,NaN,171.0,NaN
3,23.41,16.63,17.98,78.16,32.72,95.95,3.5960,14.3840,34.61,1,...,29.9,4.61,70.82,13.9,11.6,0.29,247.0,NaN,138.0,1.38
4,15.70,19.80,19.12,80.76,33.86,76.97,3.8240,15.2960,38.92,1,...,31.8,4.62,73.91,11.9,10.4,0.35,335.0,NaN,147.0,2.25


,*r-谷氨酰基转换酶,*丙氨酸氨基转换酶,*天门冬氨酸氨基转换酶,*总蛋白,*球蛋白,*碱性磷酸酶,ATSm,ATSs,feature_0_feature_1,feature_0_is_normal,...,红细胞平均血红蛋白量,红细胞计数,肌酐,血小板体积分布宽度,血小板平均体积,血小板比积,血小板计数,血糖,血红蛋白,高密度脂蛋白胆固醇
0,20.23,23.10,24.96,76.88,27.28,99.59,4.992,19.968,48.06,1,...,31.9,5.21,77.25,17.4,9.9,0.164,166.0,0.0,166.1,1.37
1,79.00,36.25,24.57,79.43,31.67,67.21,4.914,19.656,60.82,1,...,29.9,5.21,87.12,10.3,9.2,0.260,277.0,0.0,156.0,0.93
2,38.17,15.23,20.82,86.23,38.23,63.69,4.164,16.656,36.05,1,...,31.3,4.76,78.19,16.6,8.3,0.199,241.0,0.0,148.8,1.64
3,20.22,10.59,14.99,70.98,26.96,74.08,2.998,11.992,25.58,1,...,31.9,4.29,61.46,10.8,10.3,0.260,252.0,0.0,137.0,1.43
4,22.72,14.78,20.07,78.05,36.22,75.79,4.014,16.056,34.85,1,...,20.6,5.15,NaN,14.0,11.1,0.350,316.0,0.0,106.0,1.27


39.739130434782609

# 训练模型

In [54]:
params={'booster':'gbtree',
    'objective': 'binary:logistic',
    'scale_pos_weight':float(len(train["血糖"]) - sum(train["血糖"])) / sum(train["血糖"]),
    'eval_metric': 'auc',
    "subsample":0.65,
    "colsample_bytree":0.8,
    "lambda":5,
    "seed":1024,
    "silent":1,
    "verbose":0,
    "max_depth":7,
    "alpha":0.01,
    "gamma":0.6
    }

In [55]:
%%time
watchlist  = [(xgb_train,'train')]
#通过cv找最佳的nround
cv_log = xgb.cv(params,xgb_train,num_boost_round=25000,nfold=5,metrics='auc',early_stopping_rounds=50,seed=1024)

Wall time: 27 s


In [56]:
cv_log

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.699633,0.044242,0.877883,1.222921e-02
1,0.727456,0.047443,0.937588,9.955686e-03
2,0.750697,0.038615,0.962013,6.491481e-03
3,0.769997,0.022046,0.978543,3.821614e-03
4,0.773844,0.019694,0.985927,3.646891e-03
5,0.771021,0.018061,0.990490,2.056785e-03
6,0.777294,0.016661,0.992695,1.509286e-03
7,0.783773,0.015759,0.994577,8.829146e-04
8,0.784146,0.024014,0.995941,1.043785e-03
9,0.792028,0.022690,0.996750,8.771883e-04


In [57]:
bst_auc= cv_log['test-auc-mean'].max()
cv_log['nb'] = cv_log.index
cv_log.index = cv_log['test-auc-mean']
bst_nb = cv_log.nb.to_dict()[bst_auc]

In [58]:
%%time
#train
watchlist  = [(xgb_train,'train')]
model = xgb.train(params,xgb_train,num_boost_round=bst_nb+50,evals=watchlist)

[0]	train-auc:0.878387
[1]	train-auc:0.942955
[2]	train-auc:0.953937
[3]	train-auc:0.965964
[4]	train-auc:0.976418
[5]	train-auc:0.984298
[6]	train-auc:0.989177
[7]	train-auc:0.993577
[8]	train-auc:0.995795
[9]	train-auc:0.996641
[10]	train-auc:0.99723
[11]	train-auc:0.997442
[12]	train-auc:0.997855
[13]	train-auc:0.998254
[14]	train-auc:0.998659
[15]	train-auc:0.998984
[16]	train-auc:0.999368
[17]	train-auc:0.999498
[18]	train-auc:0.999563
[19]	train-auc:0.999635
[20]	train-auc:0.999696
[21]	train-auc:0.999711
[22]	train-auc:0.999777
[23]	train-auc:0.999819
[24]	train-auc:0.999893
[25]	train-auc:0.999921
[26]	train-auc:0.999946
[27]	train-auc:0.999945
[28]	train-auc:0.999946
[29]	train-auc:0.999958
[30]	train-auc:0.999974
[31]	train-auc:0.99998
[32]	train-auc:0.999983
[33]	train-auc:0.999993
[34]	train-auc:0.999992
[35]	train-auc:0.999989
[36]	train-auc:0.999993
[37]	train-auc:0.999989
[38]	train-auc:0.999999
[39]	train-auc:0.999995
[40]	train-auc:0.999999
[41]	train-auc:1
[42]	train-

In [59]:
test_pred = model.predict(xgb_test)

In [62]:
test_result = test
test_result["gt10_prob"] = test_pred
# test_result.to_csv("../classifier/gt98_prob.csv",index=None,encoding='utf-8')

In [63]:
test_result[["id","gt10_prob"]].to_csv("../classifier/gt10.csv",index=False)
test_result[test_result.gt10_prob.values > 0.499]["gt10_prob"].count()

7